# JSON to Excel

Short script to facilitate Json migration to Excel format

In [1]:
import json

with open('./BOA_database_for_exercise_from_2020.json', 'r') as f:
    boa_pharma = json.load(f)

In [ ]:
import pandas as pd

def boa_to_df(boa):

    if len(boa['opponents_reps']) == 0:
        opponents = pd.DataFrame()
    else:
        opponents = pd.concat([
        pd.DataFrame({f"Opponent {i+1}":[v[0]], f"Representative {i+1}":[v[1]]}) for i,v in enumerate(boa['opponents_reps'])
        ], axis=1)

    return pd.concat([
    pd.DataFrame.from_dict({
        "Decision date" : [boa['date']],
        "Case number" : [boa['case_number']],
        "Application number" : [boa['application_number']],
        "Publication number" : [boa['publication_number']],
        "IPC pharma" : [boa['IPC pharma']],
        "IPC biosimilar" : [boa['IPC biosimilar']],
        "IPCs" : [", ".join(boa['IPC'])],
        "Language" : [boa['lang']],
        "Title of Invention" : [boa['title_of_invention']],
        "Patent Proprietor" : [boa['patent_proprietor']],
        "Headword" : [boa['headword']],
        "Provisions" : [", ".join(boa['provisions'])],
        "Keywords" : [', '.join(boa['keywords'])],
        "Decisions cited" : [', '.join(boa['decisions_cited'])],
        "Summary" : ['\n\n'.join(boa['summary'])],
        "Decision reasons" : ['\n\n'.join(boa['decision_reasons'])],
        "Order" : [', '.join(boa['order'])],
        "Order status" : [boa['Order_status']],
        "Order status web" : [boa['Order_status_web']],
        "Order status manual" : [boa['Order_status_manual']],
        "Opponents" : [', '.join(boa['opponents'])]
    }),
    opponents
    ], axis=1)

boa_table = pd.concat([ boa_to_df(boa) for boa in boa_pharma], axis = 0)
boa_table.shape

(952, 61)

In [ ]:
# Seems that all of the columns don't have missing values, but actually they contain empty strings
boa_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 0
Data columns (total 61 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Decision date        952 non-null    object
 1   Case number          952 non-null    object
 2   Application number   952 non-null    object
 3   Publication number   952 non-null    object
 4   IPC pharma           952 non-null    int64 
 5   IPC biosimilar       952 non-null    int64 
 6   IPCs                 952 non-null    object
 7   Language             952 non-null    object
 8   Title of Invention   952 non-null    object
 9   Patent Proprietor    952 non-null    object
 10  Headword             952 non-null    object
 11  Provisions           952 non-null    object
 12  Keywords             952 non-null    object
 13  Decisions cited      952 non-null    object
 14  Summary              952 non-null    object
 15  Decision reasons     952 non-null    object
 16  Order          

In [15]:
# Replace empty strings with NaN values
boa_proc = boa_table.replace("", pd.NA)

boa_proc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 0
Data columns (total 61 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Decision date        952 non-null    object
 1   Case number          952 non-null    object
 2   Application number   952 non-null    object
 3   Publication number   949 non-null    object
 4   IPC pharma           952 non-null    int64 
 5   IPC biosimilar       952 non-null    int64 
 6   IPCs                 952 non-null    object
 7   Language             952 non-null    object
 8   Title of Invention   950 non-null    object
 9   Patent Proprietor    952 non-null    object
 10  Headword             701 non-null    object
 11  Provisions           706 non-null    object
 12  Keywords             938 non-null    object
 13  Decisions cited      952 non-null    object
 14  Summary              952 non-null    object
 15  Decision reasons     952 non-null    object
 16  Order          

In [16]:
# Define the priority order for languages
priority = {'EN': 1, 'DE': 2, 'FR': 3}

# Map the priority to a new column and sort by 'application_number' and priority
boa_proc['Priority'] = boa_proc['Language'].map(priority)
boa_proc = boa_proc.sort_values(by=['Case number', 'Priority'])

# Drop duplicates, keeping the first occurrence
boa_proc = boa_proc.drop_duplicates(subset='Case number', keep='first').drop(columns='Priority')

boa_proc.shape

(947, 61)

In [ ]:
# "Publication number" has 3 missing values but won't be used in the analysis
boa_proc['Publication number'].isna().sum()

np.int64(3)

In [ ]:
# "IPC pharma" is either 1 or 0
# 1 means that the IPC is related to pharma i.e. in the IPCs column at least one IPC is related to pharma i.e. starts with A61K
boa_proc['IPC pharma'].value_counts()

IPC pharma
1    796
0    151
Name: count, dtype: int64

In [ ]:
# "IPC biosimilar" contains only one value -> doesn't bring any useful information
boa_proc['IPC biosimilar'].value_counts()

IPC biosimilar
0    947
Name: count, dtype: int64

In [ ]:
# Contains the IPCs of the patent separated by commas
boa_proc['IPCs'].value_counts()

IPCs
A61K39/12                                                                                                                                             5
C07D277/28, C07D417/14, A61K31/427, A61P31/12                                                                                                         5
C07D277/56, A61K31/425, A61P19/06                                                                                                                     3
A61K31/00, A61K31/395, A61K31/165, A61P7/06                                                                                                           3
A61K9/70, A61K31/381                                                                                                                                  3
                                                                                                                                                     ..
A61KNone/12                                                                        

In [31]:
boa_proc['Title of Invention'].value_counts(ascending=False)

Title of Invention
Modulators of pharmacokinetic properties of therapeutics                                   4
Use of a PCV2 immunogenic composition for lessening clinical symptoms in pigs              3
Verfahren zur kontinuierlichen Herstellung von Nitrobenzol                                 3
Treatment of paroxysmal nocturnal hemoglobinuria patients by an inhibitor of complement    3
CLEANSING COMPOSITION                                                                      2
                                                                                          ..
PROCESS FOR ETHYLBENZENE PRODUCTION                                                        1
COSMETIC AND PHARMACEUTICAL FOAM                                                           1
HAIR TREATMENT COMPOSITION                                                                 1
Antigen binding proteins to proprotein convertase subtilisin kexin type 9 (PCSK9)          1
Humanized antibody against amyloid beta.           

In [37]:
boa_proc['Patent Proprietor'].value_counts(ascending=False)

Patent Proprietor
N.V. Nutricia                                                                                     25
BASF SE                                                                                           22
Société des Produits Nestlé S.A.                                                                  20
L'Oréal                                                                                           18
Novartis AG                                                                                       18
                                                                                                  ..
EUROTECNICA MELAMINE, Luxembourg, Zweigniederlassung in Ittigen                                    1
Emory University, Dana-Farber Cancer Institute, Inc., President and Fellows of Harvard College     1
R-Pharm International, LLC                                                                         1
MAP PHARMACEUTICALS, INC.                                                

In [ ]:
# Missing values are observed but they are also not available on the internet
boa_proc['Headword'].isna().sum()

np.int64(250)

In [ ]:
# Missing values are observed but they are also not available on the internet
boa_proc['Provisions'].isna().sum()

In [23]:
boa_proc.info()


<class 'pandas.core.frame.DataFrame'>
Index: 947 entries, 0 to 0
Data columns (total 61 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Decision date        947 non-null    object
 1   Case number          947 non-null    object
 2   Application number   947 non-null    object
 3   Publication number   944 non-null    object
 4   IPC pharma           947 non-null    int64 
 5   IPC biosimilar       947 non-null    int64 
 6   IPCs                 947 non-null    object
 7   Language             947 non-null    object
 8   Title of Invention   947 non-null    object
 9   Patent Proprietor    947 non-null    object
 10  Headword             697 non-null    object
 11  Provisions           702 non-null    object
 12  Keywords             933 non-null    object
 13  Decisions cited      947 non-null    object
 14  Summary              947 non-null    object
 15  Decision reasons     947 non-null    object
 16  Order          

In [45]:
boa_proc[boa_proc['Provisions'].isna()]

,Decision date,Case number,Application number,Publication number,IPC pharma,IPC biosimilar,IPCs,Language,Title of Invention,Patent Proprietor,...,Opponent 16,Representative 16,Opponent 17,Representative 17,Opponent 18,Representative 18,Opponent 19,Representative 19,Opponent 20,Representative 20
0,2020-04-30,T 0005 / 17 - 3.3.07,09781176,2323672,1,0,"A61K3/06, A61K3/12, A61P7/00",EN,COMPOSITIONS FOR THE PROPHYLAXIS AND TREATMENT...,"Despharma Egészségügyi Szolgáltató Korlátolt, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2020-10-06,T 0031 / 19 - 3.3.04,11711386,2550017,1,0,"A61K39/395, C07K16/00, C07K16/46",EN,Disulfide stabilised antibodies and fragments ...,UCB Biopharma SRL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-08-10,T 0036 / 21 - 3.3.10,11182471,2438903,1,0,"A61K8/35, A61K8/40, A61K8/49, A61Q17/04",DE,Konservierungsmittelfreies Sonnenschutzmittel,Beiersdorf AG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2020-01-14,T 0041 / 17 - 3.3.02,05797740,1797038,1,0,"C07D213/81, A61K31/44, A61P35/00",EN,THERMODYNAMICALLY STABLE FORM OF BAY 43-9006 T...,Bayer HealthCare LLC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-01-10,T 0043 / 20 - 3.3.07,08848615,2222285,1,0,"A61K9/70, A61K33/26",EN,PHARMACEUTICAL COMPOSITIONS,"Vifor Fresenius Medical Care Renal Pharma, Ltd.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-10-27,T 3002 / 19 - 3.3.04,12168565,2559440,1,0,"A61K39/395, C07K16/18, G01N33/574, C07K16/30, ...",EN,A microvesicle membrane protein and applicatio...,"Jiang, Ming-Chung",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021-12-09,T 3007 / 18 - 3.3.02,13730418,2858492,1,0,"A01N25/06, A61K8/19, A61K8/25, A61K8/26, A61K8...",EN,SPRAY FORMULATIONS WITH REDUCED CLOGGING/SEDIM...,"S.C. Johnson & Son, Inc.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-09-19,T 3030 / 19 - 3.3.02,12170826,2671450,1,0,"A01N59/16, A61K33/38, B22F1/00, B22F9/24, B82Y...",EN,"Method for preparing nanoparticles, nanopartic...",King Saud University,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-08-17,T 3034 / 19 - 3.3.10,03718790,1501775,0,0,"C07C43/11, C11D1/825",DE,ALKOXYLATGEMISCHE UND DIESE ENTHALTENDE WASCHM...,BASF SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
boa_proc.head(10)

,Decision date,Case number,Application number,Publication number,IPC pharma,IPC biosimilar,IPCs,Language,Title of Invention,Patent Proprietor,...,Opponent 16,Representative 16,Opponent 17,Representative 17,Opponent 18,Representative 18,Opponent 19,Representative 19,Opponent 20,Representative 20
0,2023-10-10,G 0002 / 22 - EBA,16160321,3056218,1,0,"A61K39/395, C07K16/18",EN,Prolongation of survival of an allograft by in...,"Alexion Pharmaceuticals, Inc.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021-06-22,G 0004 / 19 - EBA,10718590,2429542,1,0,"A61K35/74, A23L1/30, A61P37/08, A61P1/12",EN,PREVENTION AND TREATMENT OF ALLERGIC DIARRHOEA,Société des Produits Nestlé S.A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-11-23,J 0009 / 21 - 3.1.01,09007539,2100615,1,0,"A61K39/00, A61P35/00, G01N33/68, G01N33/574",EN,Cancer therapy,Biotempus Pty Ltd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-11-24,R 0005 / 22 - EBA,14172398,2801355,1,0,"A61K9/20, A61K9/50",EN,Controlled release pharmaceutical compositions...,FWP IP APS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-11-24,R 0007 / 21 - EBA,06837634,1951304,1,0,"A61K39/395, A61P19/02, A61K31/00, C07K16/28",EN,METHOD FOR TREATING JOINT DAMAGE,"F. Hoffmann-La Roche AG, Biogen Idec Inc., Gen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2020-12-10,R 0008 / 20 - EBA,05797740,1797038,1,0,"C07D13/81, A61K1/44, A61P5/00",EN,THERMODYNAMICALLY STABLE FORM OF BAY 43-9006 T...,Bayer HealthCare LLC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-05-30,R 0008 / 21 - EBA,09711390,2249859,1,0,"A61K38/09, A61P35/00, A61P35/04",EN,Treatment of metastatic stage prostate cancer ...,Ferring B.V.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-09-25,R 0010 / 20 - EBA,09011636,2168942,0,0,"C07C201/08, C07C205/06",EN,Verfahren zur kontinuierlichen Herstellung von...,Covestro Deutschland AG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-04-19,R 0014 / 22 - EBA,10705984,2400954,1,0,"A61K9/20, A61K9/28, A61K31/439",EN,PROCESS FOR FORMING SOLID ORAL DOSAGE FORMS OF...,"KRKA, d.d., Novo mesto",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021-09-28,T 0001 / 18 - 3.3.02,12703670,2670401,1,0,"C07D401/14, C07D401/12, A61K31/00, A61K31/506,...",EN,METHODS OF USING ALK INHIBITORS,Novartis AG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
